In [1]:
from os import listdir
from os.path import isfile, join
import json
import pandas as pd
import numpy as np

In [2]:
%run 'Map Extraction - Functions.ipynb'

In [3]:
GOOGLE_DIRECTORY = "/media/host_data/lions/google/"
WRITE_DIRECTORY = "/media/host_data/lions/"

In [4]:
nb_files = [ f for f in listdir(GOOGLE_DIRECTORY + 'northbound' + '/') if isfile(join(GOOGLE_DIRECTORY + 'northbound' + '/',f)) ]
sb_files = [ f for f in listdir(GOOGLE_DIRECTORY + 'southbound' + '/') if isfile(join(GOOGLE_DIRECTORY + 'southbound' + '/',f)) ]

# Checks

In [6]:
# Check for multiple-route responses
for f in nb_files:
    with open(GOOGLE_DIRECTORY + 'northbound' + '/' + f) as json_file:
        data = json.load(json_file)
        if len(data['routes']) > 1:
            print(f)
for f in sb_files:
    with open(GOOGLE_DIRECTORY + 'southbound' + '/' + f) as json_file:
        data = json.load(json_file)
        if len(data['routes']) > 1:
            print(f)

In [7]:
# Check for multiple-leg responses
for f in nb_files:
    with open(GOOGLE_DIRECTORY + 'northbound' + '/' + f) as json_file:
        data = json.load(json_file)
        if len(data['routes'][0]['legs']) > 1:
            print(f)
for f in sb_files:
    with open(GOOGLE_DIRECTORY + 'southbound' + '/' + f) as json_file:
        data = json.load(json_file)
        if len(data['routes'][0]['legs']) > 1:
            print(f)

# Run

In [8]:
northbound_durations = np.zeros(len(nb_files))
southbound_durations = np.zeros(len(sb_files))

In [14]:
nb_times =  [get_vancouver_datetime_from_filename(f, suffix='.json') for f in nb_files]
sb_times = [get_vancouver_datetime_from_filename(f, suffix='.json') for f in sb_files]

In [15]:
for idx, f in enumerate(nb_files):
    with open(GOOGLE_DIRECTORY + 'northbound' + '/' + f) as json_file:
        data = json.load(json_file)
        if 'duration_in_traffic' in data['routes'][0]['legs'][0].keys():
            northbound_durations[idx] = data['routes'][0]['legs'][0]['duration_in_traffic']['value']

In [19]:
for idx, f in enumerate(sb_files):
    with open(GOOGLE_DIRECTORY + 'southbound' + '/' + f) as json_file:
        data = json.load(json_file)
        if 'duration_in_traffic' in data['routes'][0]['legs'][0].keys():
            southbound_durations[idx] = data['routes'][0]['legs'][0]['duration_in_traffic']['value']

In [20]:
df_northbound = pd.DataFrame({'time':nb_times,'duration':northbound_durations})
df_southbound = pd.DataFrame({'time':sb_times,'duration':southbound_durations})

In [21]:
df_northbound.set_index('time', inplace=True)
df_southbound.set_index('time', inplace=True)

In [22]:
df_northbound.sort_index(inplace=True)
df_southbound.sort_index(inplace=True)

In [23]:
df_northbound.to_csv(WRITE_DIRECTORY + 'google_northbound.csv')
df_southbound.to_csv(WRITE_DIRECTORY + 'google_southbound.csv')